<a href="https://colab.research.google.com/github/pren1/VAST/blob/master/VAST_add_new_data_quickly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf VAST/
!git clone https://github.com/pren1/VAST.git
!pip3 install soundfile

Cloning into 'VAST'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 112 (delta 49), reused 59 (delta 21), pack-reused 0
Receiving objects: 100% (112/112), 44.20 MiB | 9.91 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [0]:
from VAST.util import *
import pdb
import pprint
from tqdm import tqdm
import numpy as np
import IPython
import soundfile as sf
import matplotlib.pyplot as plt
import librosa

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
'set this!'
new_vtb_name = '黑下'
target_label = 1

'should keep these consistent'
sub_size = 512
target_path = f'gdrive/My Drive/VAST/{new_vtb_name}/'
save_path = f"gdrive/My Drive/Free_RAM_VAST/{new_vtb_name}/"

In [0]:
def clip_one_second_data(seq_data, samplerate, resulted_rate = 16000):
  if len(seq_data) < samplerate:
    print("Insufficient length")
    return []
  'First, let us down sample'
  y_16k = librosa.resample(seq_data, samplerate, resulted_rate)
  clipped_res = []
  'Then, clip it'
  for index in range(0, len(y_16k), resulted_rate):
    'make sure you get the 1 seconds clipped data'
    current_clip = y_16k[index:index + resulted_rate]
    if len(current_clip) == resulted_rate:
      clipped_res.append(current_clip)
  return clipped_res

def save_split_small_batch(target_data, target_label, save_path, Batch_size = 128):
  'duplacated function can be found in VTB_classifier_playground ~'
  'save data, each npy file contains Batch_size seq of data'
  save_index = 0
  create_folders(save_path)
  for index in tqdm(range(0, len(target_label), Batch_size)):
    data_clip = target_data[index:index + Batch_size]
    label_clip = target_label[index:index + Batch_size]
    if len(data_clip) == Batch_size:
      'save data'
      save_data_array_as_npy(data_clip, save_path + f'data_{save_index}')
      save_data_array_as_npy(label_clip, save_path + f'label_{save_index}')
      save_index += 1

In [6]:
data_res = []
label_res = []

first_level_walk = [target_path + f.path.split('/')[-1] for f in os.scandir(target_path) if f.is_dir()]
for eval_target_path in first_level_walk:
  'Deal with each subpath, expect 16 in total~'
  path_list = files_in_target_folder(eval_target_path, '.flac')
  path_list = [eval_target_path + "/" + x for x in path_list]
  for single_data_path in tqdm(path_list):
    data, samplerate = sf.read(single_data_path)
    clipped_data = clip_one_second_data(data, samplerate, 8000)
    corresponding_label = [target_label] * len(clipped_data)
    if len(clipped_data) > 0:
      data_res.append(clipped_data)
      label_res.extend(corresponding_label)
'in the end, get data_res vstack'
data_res = np.vstack(data_res)
assert len(data_res) == len(label_res)

100%|██████████| 681/681 [01:08<00:00,  9.94it/s]


In [9]:
save_split_small_batch(data_res, label_res, save_path, sub_size)

100%|██████████| 6/6 [00:00<00:00, 10.88it/s]


1